<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/AM_FinBERT_finetuning_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Install libraries**

In [ ]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# **2. Load FinBERT and Prepare Dataset**

In [ ]:
# Load FinBERT tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load your labeled financial dataset
df = pd.read_csv("Financial_Sentiment_Analysis.csv")  # selected columns: 'text', 'label' (0 = negative, 1 = neutral, 2 = positive)

# Convert text labels to numerical values (if needed)
label_mapping = {"negative": 0, "neutral": 1, "positive": 2}
df['label'] = df['label'].map(label_mapping)

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Convert to Dataset format
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"],
                                   "attention_mask": train_encodings["attention_mask"],
                                   "labels": train_labels})

val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"],
                                 "attention_mask": val_encodings["attention_mask"],
                                 "labels": val_labels})

# **3. Load FinBERT and Define Training Arguments**

In [ ]:
# Load FinBERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./finbert_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# **4. Train and Save Model**

In [ ]:
# Train model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./finbert_the_quant_collective")
tokenizer.save_pretrained("./finbert_the_quant_collective")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arjt23-mitra (arjt23-mitra-fourthrev) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.356200,0.690291
2,0.558500,0.615275
3,0.356400,0.587452


('./finbert_the_quant_collective/tokenizer_config.json',
 './finbert_the_quant_collective/special_tokens_map.json',
 './finbert_the_quant_collective/vocab.txt',
 './finbert_the_quant_collective/added_tokens.json',
 './finbert_the_quant_collective/tokenizer.json')

# **5. Evaluate Model**

In [ ]:
# Load validation data
def compute_accuracy(predictions, labels):
    preds = np.argmax(predictions, axis=1)
    return np.sum(preds == labels) / len(labels)

# Get model predictions
outputs = trainer.predict(val_dataset)
accuracy = compute_accuracy(outputs.predictions, outputs.label_ids)

print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.7857


# **6. Use Fine-Tuned FinBERT for Sentiment Analysis**

In [ ]:
# Load fine-tuned model
from transformers import pipeline

finbert_pipeline = pipeline("text-classification", model="./finbert_the_quant_collective", tokenizer=tokenizer)

# Test on new financial text
test_text = "regarding deposit inflows firmwide level average deposits 3% quarter-on-quarter end-of-period deposits 2% quarter-on-quarter Ã¢â‚¬â€œ implying intra-quarter reversal recent outflow trend consequence march events estimate retained approximately $50 billion deposit inflows quarter-end"
result = finbert_pipeline(test_text)
print(result)  # Output: [{'label': 'XXXXXX', 'score': X.XX}]

Device set to use cuda:0


[{'label': 'neutral', 'score': 0.8798871040344238}]
